In [2]:
# %pip install segment-geospatial groundingdino-py leafmap localtileserver

In [1]:
import os
import leafmap
from samgeo import split_raster
from samgeo.text_sam import LangSAM

In [2]:
def find_rgb_image(laz_data_path, dop_folder):
    """
    Extract the unique identifier from a LiDAR .laz file path and find the corresponding .tif in the dop folder.

    Parameters:
        laz_data_path (str): Path to the LiDAR .laz file.
        dop_folder (str): Path to the dop folder containing .tif files.

    Returns:
        str: Path to the .jpg file if found, else None.
    """
    # Extract the unique identifier (e.g., '33470-5716') from the laz_data_path
    file_identifier = os.path.basename(laz_data_path).split('.')[0].split('_')[-1]

    # Search for a matching .tif file in the dop folder
    for root, _, files in os.walk(dop_folder):
        for file in files:
            if file_identifier in file and file.endswith('.tif'):
                return os.path.join(root, file)
    
    return None

In [3]:
def get_als_dop_matches(als_folder, dop_folder):
    """
    Match ALS files with corresponding DOP images based on their last 5-digit-4-digit tile numbers.

    Parameters:
    als_folder (str): Path to the ALS folder containing .laz files.
    dop_folder (str): Path to the DOP folder containing .tif files.

    Returns:
    dict: A dictionary where keys are ALS file paths and values are corresponding DOP image paths.
    """
    matches = {}
    # Iterate through ALS files
    for root, _, files in os.walk(als_folder):
        for file in files:
            if file.endswith('.laz'):
                # Get full path of the ALS file
                als_file_path = os.path.join(root, file)
                # Find corresponding DOP image
                dop_image_path = find_rgb_image(als_file_path, dop_folder)
                if dop_image_path:
                    matches[als_file_path] = dop_image_path
    return matches

In [4]:
def create_tiles_from_matches(matches, tile_out_dir="tiles", tile_size=(1000, 1000), overlap=0):
    """
    Create tiles for DOP images based on the matches dictionary.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and DOP image paths as values.
    tile_out_dir (str): Path to the output directory for tiles (default: "tiles").
    tile_size (tuple): Size of tiles for raster splitting (default: (1000, 1000)).
    overlap (int): Overlap between tiles (default: 0).
    """
    for als_file, dop_image in matches.items():
        # Extract file identifier from the DOP image path
        file_identifier = os.path.splitext(os.path.basename(dop_image))[0]

        # Output directory for tiles of this DOP image
        tile_dir = os.path.join(tile_out_dir, file_identifier)

        # Create tiles
        split_raster(filename=dop_image, out_dir=tile_dir, tile_size=tile_size, overlap=overlap)

        print(f"Tiles created for {file_identifier}. Saved in {tile_dir}.")

In [5]:
def apply_sam_to_tiles_from_matches(matches, tile_base_dir="tiles", mask_out_dir="masks", text_prompt="tree", box_threshold=0.24, text_threshold=0.24):
    """
    Apply LangSAM to tiles based on the matches dictionary.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and DOP image paths as values.
    tile_base_dir (str): Path to the base directory containing tiles.
    mask_out_dir (str): Path to the output directory for masks (default: "masks").
    text_prompt (str): Text prompt for LangSAM (default: "tree").
    box_threshold (float): Box threshold for LangSAM (default: 0.24).
    text_threshold (float): Text threshold for LangSAM (default: 0.24).
    """
    # Instantiate LangSAM once
    sam = LangSAM()

    for als_file, dop_image in matches.items():
        # Extract file identifier from the DOP image path
        file_identifier = os.path.splitext(os.path.basename(dop_image))[0]

        # Path to the tiles directory for this DOP image
        tile_path = os.path.join(tile_base_dir, file_identifier)

        # Output directory for masks of this tile group
        mask_dir = os.path.join(mask_out_dir, file_identifier)

        # Apply LangSAM to the tiles
        sam.predict_batch(
            images=tile_path,
            out_dir=mask_dir,
            text_prompt=text_prompt,
            box_threshold=box_threshold,
            text_threshold=text_threshold,
            mask_multiplier=255,
            dtype="uint8",
            merge=True,
            verbose=True,
        )

        print(f"Masks created for tiles in {tile_path}. Saved in {mask_dir}.")

In [6]:
def visualize_multiple_segmentation_overlays_leafmap(matches, mask_base_dir="masks", max_images=3):
    """
    Visualize multiple segmentation overlays using Leafmap for the first N images.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and DOP image paths as values.
    mask_base_dir (str): Path to the base directory containing segmentation masks.
    max_images (int): Maximum number of DOP images to visualize.
    """
    # Create a Leafmap interactive map
    m = leafmap.Map(center=[-22.1278, -51.4430], zoom=17, height="800px")

    # Add the Satellite Basemap
    m.add_basemap("SATELLITE")

    count = 0  # Counter to control the maximum number of visualizations
    for als_file, dop_image in matches.items():
        # Extract file identifier from the DOP image path
        file_identifier = os.path.splitext(os.path.basename(dop_image))[0]

        # Path to the segmentation mask for this DOP image
        segmentation_tif_path = os.path.join(mask_base_dir, file_identifier, "merged.tif")

        # Check if the segmentation file exists
        if not os.path.exists(segmentation_tif_path):
            print(f"Segmentation mask not found for {file_identifier}. Skipping visualization.")
            continue

        # Add the original DOP image as a raster layer
        try:
            m.add_raster(dop_image, layer_name=f"DOP Image ({file_identifier})")
        except Exception as e:
            print(f"Error adding DOP image to map for {file_identifier}: {e}")
            continue

        # Add the segmentation mask as an overlay layer
        try:
            m.add_raster(segmentation_tif_path, layer_name=f"Segmentation Mask ({file_identifier})", opacity=0.5)
        except Exception as e:
            print(f"Error adding segmentation mask to map for {file_identifier}: {e}")
            continue

        print(f"Added DOP and segmentation mask for {file_identifier} to the map.")

        # Break after visualizing the maximum number of images
        count += 1
        if count >= max_images:
            break

    print(f"Displaying map with {count} segmentation overlays.")
    return m  # Return the map to visualize the images


In [7]:
# Set ALS and DOP folder paths'
als_folder = "data/Tschernitz/als"
dop_folder = "data/Tschernitz/dop"

# Get matches
als_dop_matches = get_als_dop_matches(als_folder, dop_folder)

# Print matches
for als_file, dop_image in als_dop_matches.items():
    print(f"ALS File: {als_file} -> DOP Image: {dop_image}")

ALS File: data/Tschernitz/als\als_33470-5714\als_33470-5714.laz -> DOP Image: data/Tschernitz/dop\dop_33470-5714\dop_33470-5714.tif
ALS File: data/Tschernitz/als\als_33470-5715\als_33470-5715.laz -> DOP Image: data/Tschernitz/dop\dop_33470-5715\dop_33470-5715.tif
ALS File: data/Tschernitz/als\als_33470-5716\als_33470-5716.laz -> DOP Image: data/Tschernitz/dop\dop_33470-5716\dop_33470-5716.tif


In [ ]:
# Define output directory for tiles
tile_output_directory = "data/Tschernitz/tiles"

# Create tiles using the matches dictionary
#create_tiles_from_matches(als_dop_matches, tile_out_dir=tile_output_directory,  tile_size=(1000, 1000), overlap=0)

In [ ]:
text_prompt="tree"

# Define output directory for masks
mask_output_directory = f"data/Tschernitz/ground_truth_masks/{text_prompt}_masks"

In [ ]:
# Apply SAM using the matches dictionary
apply_sam_to_tiles_from_matches(als_dop_matches, tile_base_dir=tile_output_directory, mask_out_dir=mask_output_directory)

In [ ]:
# Visualize multiple segmentation overlays using Leafmap
leafmap_instance = visualize_multiple_segmentation_overlays_leafmap(als_dop_matches, mask_base_dir=mask_output_directory, max_images=3)
leafmap_instance